In [2]:
import pymongo
from pymongo import MongoClient

import pandas as pd

import numpy as np

In [3]:
conn=MongoClient('mongodb://localhost:27017//pds')
db=conn.pds

# First Dataset -Lemme think of a name

### Basic pay by age data

In [3]:
df=pd.read_excel("dataset/mrsd_2017Wages_table3.xlsx",'T3',index_col=2)
column_names=df.iloc[4:6,2:-5].fillna(method="pad",axis=1).values

#make column names
col_name=[]
for i in range(column_names[0].size):
    s=str(column_names[0][i])+" "+str(column_names[1][i]).replace("nan","").replace("\n","")
    col_name.append(s)

    

data=df.iloc[9:,2:-5]
data.columns=col_name


col_name.append("Category")
for i in range(data.shape[0]):
    data.loc[:,'Category']=np.nan

data=data.apply(pd.to_numeric, errors='coerce')


a=data.isnull().all(axis=1)
a.columns=['a']

for i in range(a.shape[0]):
    if a[i]==True:
        data.iloc[i,11]=data.index[i]

data[['Category']]=data[['Category']].fillna(method='pad')

data=data.dropna(subset=col_name[1:-1],how="all")
data=data.drop([col_name[0],col_name[2],col_name[3],col_name[5],col_name[6],col_name[8],col_name[9]],axis=1)
print(data)

                                                    25  -  29  Years Basic Wage ($)  \
Administration manager                                                       4000.0   
Advertising/Public relations manager                                         3800.0   
Arts and cultural centre manager (including gal...                              NaN   
Budgeting and financial accounting manager (inc...                           5500.0   
Building and construction project manager                                    3776.0   
Business development manager                                                 4254.0   
Chief information officer/Chief technology officer                              NaN   
Chief operating officer/General Manager                                         NaN   
Child care services manager                                                     NaN   
Customer service manager                                                        NaN   
Education and training institution manager 

### 25th, Median, 75th percentile of pay by industry

In [6]:
df2=pd.read_excel("dataset/mrsd_2017Wages_table4.xlsx",'T4',usecols=[2,4,5,6],index_col=0,skiprows=6)
df2=df2.dropna()
df_merged=pd.merge(data,df2,how="outer", left_index=True, right_index=True)
print(df_merged.Category.unique())

['PROFESSIONALS' 'CLERICAL SUPPORT WORKERS'
 'ASSOCIATE PROFESSIONALS AND TECHNICIANS' 'MANAGERS' nan
 'CLEANERS, LABOURERS AND RELATED WORKERS'
 'CRAFTSMEN AND RELATED TRADES WORKERS' 'SERVICE AND SALES WORKERS'
 'PLANT AND MACHINE OPERATORS AND ASSEMBLERS']


### Put merged dataframe to mongo

In [6]:
col_payByIndustry=db.PayByIndustry
# df_merged=df_merged.reset_index()  <-- dun run this line again
df_merged=df_merged.dropna()
print(df_merged)
col_payByIndustry.insert_many(df_merged.to_dict("records"))

                                                 index  \
0                Accountant (excluding tax accountant)   
1                     Accounting and bookkeeping clerk   
2    Accounting associate professional (eg assistan...   
3                               Administration manager   
4                 Advertising/Public relations manager   
13                     Applications/Systems programmer   
17                      Assistant electronics engineer   
18                    Assistant manufacturing engineer   
19                       Assistant mechanical engineer   
26                       Beautician and related worker   
35   Broadcasting technician and audio-visual operator   
36   Budgeting and financial accounting manager (in...   
37           Building and construction project manager   
38                         Building and fire inspector   
41                                          Bus driver   
42                      Business development executive   
43            

## Starting pay by course cluster

In [4]:
xl=pd.ExcelFile("dataset/Gross_Starting_Salary_of_Graduates.xlsx")
df5=pd.read_excel(xl,'University',usecols=[1,5,6],skiprows=2)
df5=df5.dropna(how="all")
df5=df5.fillna(method="pad")
df5=df5.drop(0)
df5=df5.rename(columns={"Unnamed: 0":"Course Cluster"})
df5['Category']='University'
# print(df5)

df6=pd.read_excel(xl,'Polytechnic',usecols=[1,5,6],skiprows=2)
df6=df6.dropna(how="all")
df6=df6.fillna(method="pad")
df6=df6.drop(0)
df6=df6.rename(columns={"Unnamed: 0":"Course Cluster"})
df6['Category']='Polytechnic'
# print(df6)

df_56=pd.concat([df5,df6])
df_56=df_56.sort_values(by=["Category","Course Cluster"])

print(df_56)


# col_startingPay=db.StartingPay
# col_startingPay.insert_many(df5.to_dict("records"))

                               Course Cluster  Employment Rate (%)  \
17                           Applied Sciences                 55.2   
18                           Applied Sciences                 56.3   
19                           Applied Sciences                 39.8   
20                           Applied Sciences                 59.0   
4   Built Environment, Engineering & Maritime                 61.8   
5   Built Environment, Engineering & Maritime                 47.8   
6   Built Environment, Engineering & Maritime                 68.8   
7   Built Environment, Engineering & Maritime                 54.9   
8   Built Environment, Engineering & Maritime                 53.6   
9   Built Environment, Engineering & Maritime                 85.1   
10  Built Environment, Engineering & Maritime                 67.2   
11  Built Environment, Engineering & Maritime                 56.1   
12  Built Environment, Engineering & Maritime                 57.7   
13  Built Environmen

In [28]:
import pygal
from pygal.style import Style

client = MongoClient()

db = client.pds
collection = db.StartingPay

cursor = collection.find()

# Expand the cursor and construct the DataFrame
df6 =  pd.DataFrame(list(cursor))
df6=df6.drop(48)
# print(df6)

custom_style = Style(
  background='transparent')

box_plot = pygal.Box(fill=True, interpolate='cubic', style=custom_style,legend_at_bottom=True,height=700,legend_box_size=18)

for crse, df_split in df6.groupby('Course Cluster'):
    box_plot.add(crse, df_split["Median Monthly Gross Starting Salary ($)"].values)
    
# pygal.config(disable_xml_declaration)
box_plot.render_in_browser()
# box_plot.render(is_unicode=True)

file://C:/Users/clair/AppData/Local/Temp/tmpinkks3c7.html


## Household Expenditure

In [41]:
df4=pd.read_csv("dataset/average-mhe-by-type-of-goods-and-services-broad-and-expenditure-quintile.csv",usecols=[1,2,3],skiprows=18)
df4.columns=["Expenditure Quintile","Category","Value"]
df4=df4.sort_values(by=["Expenditure Quintile","Category"])
print(df4.Category.unique())

df4=df4.loc[df4['Category'].isin(["Clothing and Footwear","Communication","Food and Non-Alcoholic Beverages","Health",
                                 "Housing and Utilities","Transport"])]
print(df4.Category.unique())
col_householdExpenditure=db.HouseholdExpenditure
col_householdExpenditure.insert_many(df4.to_dict("records"))

['Accommodation Services' 'Alcoholic Beverages and Tobacco'
 'Clothing and Footwear' 'Communication' 'Educational Services'
 'Food Serving Services' 'Food and Non-Alcoholic Beverages'
 'Furnishings, Household Equipment and Routine Household Maintenance'
 'Health' 'Housing and Utilities'
 'Imputed rentals for owner-occupied accommodation'
 'Miscellaneous Goods and Services' 'Non-Assignable Expenditure'
 'Number of Resident Households' 'Recreation and Culture' 'Total'
 'Total, including imputed rental of owner-occupied accommodation'
 'Transport']
['Clothing and Footwear' 'Communication'
 'Food and Non-Alcoholic Beverages' 'Health' 'Housing and Utilities'
 'Transport']


## Car Price (June 2018)

In [6]:
xl = pd.ExcelFile("dataset/Car_Cost_Update.xls")
skip = xl.book.sheet_by_index(0).nrows-289
df1=pd.read_excel(xl,usecols=[1,2,4,16],skiprows=[1,2],skip_footer=skip)
df1.columns=['Brand','Model','Engine Capacity (cc)','Basic Cost with COE']
print(df1)

col_carPrice=db.CarPrice
col_carPrice.insert_many(df1.to_dict("records"))

            Brand                                    Model  \
0      ALFA ROMEO                         GIULIA 2.0L BASE   
1      ALFA ROMEO                              GIULIA 2.0L   
2      ALFA ROMEO                        GIULIA 2.0L SUPER   
3    ASTON MARTIN                                  DB11 V8   
4            AUDI         A3 SEDAN 1.0 TFSI S TRONIC (LED)   
5            AUDI     A3 SPORTBACK 1.0 TFSI S TRONIC (LED)   
6            AUDI         A4 SEDAN 2.0 TFSI S TRONIC (NAV)   
7            AUDI               A4 SEDAN 2.0 TFSI S TRONIC   
8            AUDI           A5 CABRIOLET 2.0 TFSI (DESIGN)   
9            AUDI         A5 SB 2.0 TFSI S TRONIC (DESIGN)   
10           AUDI             A6 1.8 TFSI ULTRA (PI) (NAV)   
11           AUDI                     Q2 1.0 TFSI S TRONIC   
12           AUDI                     Q3 1.4 TFSI S TRONIC   
13           AUDI            Q5 SPORT 2.0 TFSI QU S TRONIC   
14           AUDI                 Q7 2.0 TFSI QU (252 BHP)   
15      

## Page footer (ULPT from reddit)

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

col_ULPT= db.ULPT

url="https://www.reddit.com/r/UnethicalLifeProTips/top/?t=all"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}

response=requests.get(url,headers=headers)
content = response.text
# print(content)
soup=bs(content,"html.parser")

r=soup.find_all('h2')
ulpt= [R.text.strip() for R in r]
ulpt=[{"ulpt": R.replace("ULPT: ","")} for R in ulpt]
print (ulpt)

col_ULPT.insert_many(ulpt)